In [7]:
from gibbs.dataclass import GibbsResult,get_results
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm,trange
import plotly.express as px
from gibbs.learning.bayesian_learning import BayesianLearning
from gibbs.learning.constraint_matrix import ConstraintMatrixFactory
from qiskit.quantum_info import Statevector
from gibbs.learning.klocal_pauli_basis import KLocalPauliBasis
from scipy.linalg import block_diag
from scipy.sparse import bmat 
from qiskit.quantum_info import state_fidelity,Statevector
from gibbs.utils import number_of_elements, simple_purify_hamiltonian, spectral_dec,printarray, expected_state,identity_purification
from gibbs.plots.plots import preparation, compare_preparations, preparation_error
# plt.rcParams['text.usetex'] = True

%load_ext autoreload
%autoreload 2

qasm_result,classic_result = get_results("../scripts_lap/job_14_qasm_ising")
print(classic_result.date,qasm_result.date)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
compare_preparations([classic_result,qasm_result],["Classic","Qasm"],-1).show()
idnty_pur = identity_purification(qasm_result.num_qubits)
df = {
    "Betas" : qasm_result.betas,
    "Qasm Fidelity" : qasm_result.fidelity_evolution(),
    "Classic Fidelity" : classic_result.fidelity_evolution(),
    "Identity Fidelity" : [ state_fidelity(simple_purify_hamiltonian(b*qasm_result.hamiltonian),idnty_pur) for b in qasm_result.betas],
}
px.line(df,x="Betas",y=["Qasm Fidelity","Classic Fidelity","Identity Fidelity"],title="Fidelity Evolution")

In [8]:
long_time_result = get_results("../saved_simulations/turbo/really_long_evolution")[0]
idnty_pur = identity_purification(long_time_result.num_qubits)
df = {
    "Betas" : long_time_result.betas,
    "Classic Fidelity" : long_time_result.fidelity_evolution(True),
    "Identity Fidelity" : [ state_fidelity(simple_purify_hamiltonian(b*long_time_result.hamiltonian),idnty_pur) for b in tqdm(long_time_result.betas)],
}

100%|██████████| 501/501 [00:59<00:00,  8.36it/s]


In [10]:
px.line(df,x="Betas",y=["Classic Fidelity","Identity Fidelity"],title="Fidelity Evolution")

In [ ]:
qasm_result.animated_hamiltonian(func=np.real)

In [ ]:
timestep = -1
df ={
    "Error": np.real(qasm_result.cfaulties[timestep]-qasm_result.coriginal[timestep]*qasm_result.betas[timestep]),
    "Classic Error": np.real(classic_result.cfaulties[timestep]-classic_result.coriginal[timestep]*classic_result.betas[timestep]),
     }
# df["Error"] = df["Error"][np.abs(df["Error"])>1e-4]
# df["Classic Error"] = df["Classic Error"][np.abs(df["Classic Error"])>1e-4]

px.histogram(df,x=["Error","Classic Error"],nbins=100,barmode='overlay',histnorm="probability")